In [9]:
import os
import json
from datetime import datetime
from itertools import product
from tqdm import tqdm
import numpy as np
import xarray as xr
import xesmf as xe
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import cartopy as cart
import cartopy.crs as ccrs
import dask
from dask.diagnostics import ProgressBar
from dask.distributed import Client
from dask_jobqueue import PBSCluster

from src.attrs import PATHS, GLOBALS
from src import utils
from src.data import loading

In [10]:
FILE = 'compute_sea_level_vars.ipynb'

In [11]:
def compute_zos(ssh_2, areas):
    # Compute GMSL and zos
    gmsl = ssh_2.weighted(areas).mean(dim=('lat', 'lon'))
    zos = ssh_2 - gmsl
    
    # Convert to meters
    zos = zos / 100
    
    # Units
    zos.name = 'zos'
    zos.attrs['long_name'] = 'dynamic_sea_level'
    zos.attrs['units'] = 'm'
    zos.attrs['history'] = (
        f"Created on {datetime.now()} using {FILE}"
    )
    
    return zos

def compute_effective_sea_level(zos, psl, areas, mask):    
    # Spatial anomaly over ocean
    psl_ocn = xr.where(mask, psl, np.nan)
    p_avg = psl_ocn.weighted(areas).mean(dim=('lat','lon'))
    p_anom = psl_ocn - p_avg
    
    # Inverse barometer contribution
    eta_ib = - p_anom / (GLOBALS['rho_ref'] * GLOBALS['gravity'])
    eta_ib.attrs['units'] = 'm'
    
    # Effective sea level
    ssh = zos + eta_ib
    
    # Add metadata
    ssh.name = 'SSH'
    ssh.attrs['units'] = 'm'
    ssh.attrs['long_name'] = 'effective_sea_level'
    ssh.attrs['description'] = (
        "Effective sea level, including the inverse-barometer "
        "contribution to sea level."
    )
    ssh.attrs['history'] =  (
        f"Created on {datetime.now()} using {FILE}"
    )
    
    return ssh

In [4]:
cluster = PBSCluster(memory='64GB')
client = Client(cluster)

2024-01-30 10:06:26,686 - distributed.diskutils - ERROR - Failed to remove '/tmp/dask-scratch-space/scheduler-ljv0_5zq' (failed in <built-in function lstat>): [Errno 2] No such file or directory: '/tmp/dask-scratch-space/scheduler-ljv0_5zq'


In [5]:
cluster.scale(4)
cluster

Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/abrettin/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://128.117.208.112:41469,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/abrettin/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [1]:
def main():
    utils.log("Begin script")
    START_TIME = datetime.now()

    for member, init_year in product(GLOBALS['members'], GLOBALS['init_years']):
        utils.log(f"LE-{init_year}.{member}", START_TIME)

        # Load SSH_2, PSL, areas, mask
        ssh_2 = loading.load_dataset('SSH_2', init_year, member)
        psl = loading.load_dataset('PSL', init_year, member)
        areas = loading.load_areas()
        mask = loading.load_land_mask()

        # Compute zos and write to file
        utils.log(f"Saving zos to {PATHS['sea_level']}", START_TIME)
        zos = compute_zos(ssh_2, areas)
        zos_ds = xr.Dataset({'zos': zos})

        zos_ds.persist()
        zos_ds.to_zarr(
            os.path.join(
                PATHS['sea_level'],
                f'LE2-{init_year}.{member}.zos.zarr'
            )
        )

        # Compute inverse barometer component to sea level
        utils.log(f"Computing effective sea level for {init_year}.{member}", START_TIME)
        with dask.config.set(**{'array.slicing.split_large_chunks': False}):
            ssh = compute_effective_sea_level(zos, psl, areas, mask)
        ssh_ds = xr.Dataset({'SSH': ssh})

        utils.log(f"Saving SSH to {PATHS['sea_level']}", START_TIME)
        ssh_ds = ssh_ds.chunk({'time': 3650})
        ssh_ds.persist()
        ssh_ds.to_zarr(
            os.path.join(
                PATHS['sea_level'],
                f'LE2-{init_year}.{member}.SSH.zarr'
            )
        )

        utils.log(f"Saved effective sea level for {init_year}.{member}", START_TIME)
        print("Restarting client")
        client.restart()

    utils.log("PROCESS_COMPLETED", START_TIME)
    print(datetime.now() - START_TIME)
    
    return 0

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [12]:
main()

2024-01-30 09:20:51.298507	 Begin script
0:00:00.000013	 LE-1251.011
0:00:07.904265	 Saving zos to /glade/scratch/abrettin/sea_level/
0:00:43.328043	 Computing effective sea level for 1251.011
0:00:43.812145	 Saving SSH to /glade/scratch/abrettin/sea_level/
0:04:52.183302	 Saved effective sea level for 1251.011
Restarting client
0:04:56.824757	 LE-1281.011
0:05:08.199200	 Saving zos to /glade/scratch/abrettin/sea_level/
0:05:35.979202	 Computing effective sea level for 1281.011
0:05:36.593900	 Saving SSH to /glade/scratch/abrettin/sea_level/
0:09:41.772340	 Saved effective sea level for 1281.011
Restarting client
0:09:46.760456	 LE-1301.011
0:09:59.902689	 Saving zos to /glade/scratch/abrettin/sea_level/
0:10:29.484492	 Computing effective sea level for 1301.011
0:10:30.108281	 Saving SSH to /glade/scratch/abrettin/sea_level/
0:14:40.125386	 Saved effective sea level for 1301.011
Restarting client
0:14:44.489934	 LE-1251.012
0:14:56.605504	 Saving zos to /glade/scratch/abrettin/sea_lev

0

In [13]:
cluster.close()
client.close()